In [2]:
import pandas as pd
import numpy as np

import re
from tqdm import tqdm



# Processing

In [8]:
df = pd.read_csv('data/main_df.csv')
df.head()

,Name,Surah,Ayat,Arabic,Translation1,Translation2,Translation3,Tafaseer1,Tafaseer2
0,The Opening,1,1,بِسمِ ٱلله الرَّحْمٰنِ الرَّحِيـمِ,"All praise be to Allah alone, the Sustainer of...","In the Name of God, the Merciful, the Compassi...","In the name of Allah, the Beneficent, the Merc...",In the Name of God the Compassionate the Merciful,"In the name of Allah, the Beneficent, the Merc..."
1,The Opening,1,2,ٱلْحَمْدُ للَّهِ رَبِّ ٱلْعَالَمِينَ,"Most Compassionate, Ever-Merciful,","Praise belongs to God, the Lord of all Being,","Praise be to Allah, Lord of the Worlds,",In the Name of God the name of a thing is that...,And on his authority it is related that Ibn 'A...
2,The Opening,1,3,ٱلرَّحْمـٰنِ ٱلرَّحِيمِ,Master of the Day of Judgment.,"the All-merciful, the All-compassionate,","The Beneficent, the Merciful.",The Compassionate the Merciful that is to say ...,(The Beneficent) the Gentle. (The Merciful) th...
3,The Opening,1,4,مَـٰلِكِ يَوْمِ ٱلدِّينِ,(O Allah!) You alone do we worship and to You ...,the Master of the Day of Doom.,"Owner of the Day of Judgment,",Master of the Day of Judgement that is the day...,(Owner of the Day of Judgement) the Arbitrator...
4,The Opening,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ,"Show us the straight path,",Thee only we serve; to Thee alone we pray for ...,Thee (alone) we worship; Thee (alone) we ask f...,You alone we worship and You alone we ask for ...,"(Thee (alone) we worship), we turn to you as t..."


In [9]:
arabic = []
text=[]
for index, row in df.iterrows():
    arabic.append(row['Arabic'])
    t = ""
    t += row['Name'] + "|" + str(row['Arabic'])+"|"+ str(row['Surah'])+"|"+str(row['Ayat'])+"|"
    for j in range(1, 4):
        t += row['Translation' + str(j)] + ";"
    t += "|"
    for j in range(1, 3):
        t+= row['Tafaseer' + str(j)] + ";"
    t = t[:-1]
    text.append(t)

In [10]:
text[0]

'The Opening|1|1|All praise be to Allah alone, the Sustainer of all the worlds,;In the Name of God, the Merciful, the Compassionate;In the name of Allah, the Beneficent, the Merciful.;|In the Name of God the Compassionate the Merciful;In the name of Allah, the Beneficent, the Merciful.'

In [11]:
arabic[0]

'بِسمِ ٱلله الرَّحْمٰنِ الرَّحِيـمِ'

In [14]:
df_flatten = [' '.join(val[:2048]) for val in df.astype(str).values.tolist()]
len(df_flatten)

6236

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Preprocess the dataset
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(text)

# Define the search function
def search(query, top_k=5):
    query_vec = vectorizer.transform([query])
    sim_scores = cosine_similarity(query_vec, X).flatten()
    top_indices = sim_scores.argsort()[::-1][:top_k]
    top_paragraphs = [text[i] for i in top_indices]
    
    
    
    return top_paragraphs

In [26]:
x = search("inheritance")

In [29]:
x[0].split("|")

['Smoke',
 '44',
 '28',
 'So it happened! We made another people the inheritors of all that.;Even so; and We bequeathed them upon another people.;Even so (it was), and We made it an inheritance for other folk;;',
 'So it was kadhālika is the predicate of a subject that is missing such as ‘the matter was so’ and We made these an inheritance that is to say their riches for another people namely the Children of Israel.;(Even so (it was)) We destroyed them, (and We made it an inheritance for other folk) We made the Children of Israel inherit all that after they were destroyed;']